# Dataanalyse

Her analyserer temperaturdata, klimagassutslipp og luftkvalitetsdata ved bruk av funksjoner i `statistikk.py`.


#### Importere nødvendige biblioteker og moduler


In [ ]:
import pandas as pd
import numpy as np

import sys, os
sys.path.append(os.path.abspath("../src"))


import importlib
import statistikk
importlib.reload(statistikk)

## Analyse av temperaturdata

I denne seksjonen blir historiske temperaturdata fra Gløshaugen analysert.

### Statistikk for tempraturen hver måned og årlig gjennomsnitt 

In [ ]:

from statistikk import analyser_temperatur

resultat = analyser_temperatur("../data/temp_gloshaugen_historisk_renset_ 50.csv")


for navn, data in resultat.items():
    print("Statistikk for tempraturen hver måned")
    print(data["samlet_statistikk"])
    if data["årlig_snitt"] is not None:
        print("Årlig gjennomsnitt:")
        print(data["årlig_snitt"])
årlig_snitt = resultat["temperatur"]["årlig_snitt"]


### Endring i gjennomsnittstemperatur fra første til siste år

In [ ]:
from statistikk import beregn_endring_totalt

# Endring i gjennomsnittstemperatur fra første til siste år
resultat = beregn_endring_totalt(årlig_snitt)
print(f"Endring i temperatur fra {resultat['startår']} til {resultat['sluttår']}:")
print(f"{resultat['endring']:.2f} °C ({resultat['prosent']:.2f} %)")


### Årlig endring i temperatur

In [ ]:
from statistikk import beregn_endring_årlig

endringsdata = beregn_endring_årlig(årlig_snitt, verdikolonne='årsgjennomsnitt')
display(endringsdata.round(2))


### Høyeste og laveste gjennomsnittstemperatur

In [ ]:
from statistikk import ekstremverdier

ekstremer = ekstremverdier(årlig_snitt, verdikolonne='årsgjennomsnitt')

print(f"Høyeste år: {ekstremer['maksimum']['år']}. Da var det i gjennomsnitt: {ekstremer['maksimum']['verdi']:.2f} °C")
print(f"Laveste år: {ekstremer['minimum']['år']}. Da var det i gjennomsnitt: {ekstremer['minimum']['verdi']:.2f} °C")

### Gjenomsnittet for hvert tiår

In [ ]:
from statistikk import tiår_snitt

tiår_snitt = tiår_snitt(årlig_snitt, verdikolonne='årsgjennomsnitt')
display(tiår_snitt.round(2))


### Temperaturavvik per år

In [ ]:
from statistikk import beregn_avvik

avvik_df = beregn_avvik(årlig_snitt)
display(avvik_df.round(2))


## Analyse av klimagassutslipp i verden

Denne seksjonen analyserer globale utslippsdata.


#### Henter inn datasettet med globale klimagassutslipp

In [ ]:
statistikk_verden, df_total_verden, df_verden = statistikk.analyser_fil("../data/klimagassutslipp_verden_renset.csv",datokolonne="År",groupby="år")

### Utslipp per tiår i hele verden

In [ ]:
from statistikk import tiår_snitt

tiårs_snitt = tiår_snitt(df_verden, verdikolonne='utslipp_i_co2_ekvivalenter')

print("Utslipp per tiår globalt:")
display(tiårs_snitt)

### Året med høyest og lavest utslipp i verden

In [ ]:
from statistikk import ekstremverdier

ekstremer = ekstremverdier(df_verden, verdikolonne='utslipp_i_co2_ekvivalenter')

print(f"Høyeste utslipp: {ekstremer['maksimum']['år']} med {ekstremer['maksimum']['verdi']:.2f}")
print(f"Laveste utslipp: {ekstremer['minimum']['år']} med {ekstremer['minimum']['verdi']:.2f}")


### Endring i klimagassutslipp fra første til siste år

In [ ]:
from statistikk import beregn_endring_totalt

resultat_utslipp = beregn_endring_totalt(df_verden, verdikolonne='utslipp_i_co2_ekvivalenter')

print(f"Endring i utslipp fra {resultat_utslipp['startår']} til {resultat_utslipp['sluttår']}:")
print(f"{resultat_utslipp['endring']:.2f} ({resultat_utslipp['prosent']:.2f} %)")


### Årlig endring i klimagassutslipp

In [ ]:
from statistikk import beregn_endring_årlig

endringsdata = beregn_endring_årlig(df_verden, verdikolonne='utslipp_i_co2_ekvivalenter')
display(endringsdata.round(2))


## Analyse av klimagassutslipp i Norge

Denne seksjonen analyserer historisk klimagassutslipp i Norge.

#### Leser inn data om Norges klimagassutslipp fra en ferdig renset CSV-fil

In [ ]:
statistikk_norge, df_total_norge, df_norge = statistikk.analyser_fil("../data/klimagassutslipp_norge_renset.csv", sep=",", datokolonne="år", groupby="år")

### Utslipp i Norge per kilde og år

In [ ]:
from statistikk import analyser_utslipp_norge

stats_per_kilde_år = analyser_utslipp_norge(df_norge)

if stats_per_kilde_år is not None:
    display(stats_per_kilde_år)


### Gjennomsnitts utslipp og median for hver kilde totalt i Norge

In [ ]:
from statistikk import gjennomsnitt_per_kilde

stats_per_kilde = gjennomsnitt_per_kilde(df_norge)
display(stats_per_kilde)


### Endring av utslipp i norge fra startår til sluttår 

In [ ]:
from statistikk import lag_totaldata, beregn_endring_totalt

total = lag_totaldata(df_total_norge)
resultat = beregn_endring_totalt(total, årskolonne='år', verdikolonne='utslipp')

print(f"Endring i utslipp fra {resultat['startår']} til {resultat['sluttår']}:")
print(f"{resultat['endring']:.0f} ({resultat['prosent']:.2f} %)")


### Året med høyest og lavest utslipp i Norge

In [ ]:
from statistikk import ekstremverdier

# Filtrer ut "Alle kilder"
df_alle_kilder = df_total_norge[df_total_norge['kilde_(aktivitet)'].str.lower().str.contains("alle kilder")].copy()
ekstremer = ekstremverdier(df_alle_kilder, verdikolonne='utslipp_til_luft_(1_000_tonn_co2-ekvivalenter,_ar5)')

print(f"Høyeste utslipp: {ekstremer['maksimum']['år']} med {ekstremer['maksimum']['verdi']:.0f} (1 000 tonn CO₂-ekv.)")
print(f"Laveste utslipp: {ekstremer['minimum']['år']} med {ekstremer['minimum']['verdi']:.0f} (1 000 tonn CO₂-ekv.)")


### Årlig endring for hver kilde i Norge

In [ ]:
from statistikk import beregn_endring_per_kilde

endringer = beregn_endring_per_kilde(stats_per_kilde_år)

for kilde, df_endring in endringer.items():
    print(f"\nEndring for kilde: {kilde}")
    display(df_endring.round(2))


### Gjennomsnittlig utslipp per tiår i Norge (alle kilder)

In [ ]:
from statistikk import tiår_snitt

kol_utslipp = [kol for kol in df_alle_kilder.columns if "utslipp" in kol.lower() and "co2" in kol.lower()][0]
tiårsdata_norge = tiår_snitt(df_alle_kilder, verdikolonne=kol_utslipp)

display(tiårsdata_norge)


## Analyse av luftkvalitet

I denne seksjonen analyseres utviklingen av luftkvaliteten på Gløshaugen i Trondheim

#### Leser inn data for historisk luftkvalitet fra CSV-fil

In [ ]:
statistikk_luft, df_total, df_luft = statistikk.analyser_fil("../data/gyldig_historisk_luftkvalitet.csv", datokolonne="tid", groupby="år")

### Statistikk for luftkvalitetsmålinger

In [ ]:
for stoff, df in statistikk_luft.items():
    print(f"Statistikk: {stoff}")
    display(df[['år', 'gjennomsnitt', 'median']].round(2))

### Årlige og månedlige trender i luftkvalitet

In [ ]:
from statistikk import analyser_luftkvalitet, legg_til_tid

df_luft = legg_til_tid(df_luft)
målinger = [kol for kol in df_luft.columns if "ugm3" in kol]

resultater = analyser_luftkvalitet(df_luft, målinger)

for stoff, data in resultater.items():
    print(f"Analyse for: {stoff}")
    print("Årlig snitt:")
    display(data["årlig"].round(2))

    print("\nMånedlig snitt:")
    display(data["månedlig"].round(2))

    print("\nEkstremverdier:")
    print(f"Maks: {data['ekstremer']['maksimum']['verdi']:.2f} µg/m³ på {data['ekstremer']['maksimum']['år']}")
    print(f"Min: {data['ekstremer']['minimum']['verdi']:.2f} µg/m³ på {data['ekstremer']['minimum']['år']}")
